In [ ]:
import numpy as np
import tensorflow as tf
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Path to frozen detection graph
PATH_TO_MODEL_DIR = 'C:\Users\drish\models'
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

# Path to label map
PATH_TO_LABELS = 'C:\Users\drish\models'

# Load the model
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Function to perform object detection on a frame
def detect_objects(image_np):
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = detect_fn(input_tensor)

    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    return detections

# Load an image
image_path = 'C:\Users\drish\models/d_image.jpg'
image_np = cv2.imread(image_path)

# Perform object detection
detections = detect_objects(image_np)

# Visualize the results
image_np_with_detections = image_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=.30,
    agnostic_mode=False
)

# Display the results
cv2.imshow('Object Detection', cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()